# Pdf Q&A

In [1]:
# %pip install pypdf
# %pip install chromadb

In [2]:
from langchain.document_loaders import PyPDFLoader

### pdf loading

In [3]:
loader = PyPDFLoader("/Volumes/storage/patent/2023/20230102/1020150020108/B012/PDF/1020150020108.pdf")

In [4]:
document = loader.load()

In [5]:
document[0].page_content[:800]

'(19) 대한민국특허청(KR)\n(12) 등록특허공보(B1)(45) 공고일자   2023년01월02일\n(11) 등록번호   10-2483612\n(24) 등록일자   2022년12월28일\n(51) 국제특허분류(Int. Cl.)\n     H05K 1/18 (2006.01)  H05K 3/00 (2019.01)\n     H05K 3/46 (2006.01)\n(52) CPC특허분류\n     H05K 1/186 (2013.01)\n     H05K 3/0047 (2013.01)\n(21) 출원번호       10-2015-0020108\n(22) 출원일자       2015년02월10일\n     심사청구일자   2020년01월28일 \n(65) 공개번호       10-2016-0097800\n(43) 공개일자       2016년08월18일\n(56) 선행기술조사문헌\nWO2014193641 A1*\nUS20120037411 A1\nKR1020070029930 A\nKR1020070078986 A\n*는 심사관에 의하여 인용된 문헌(73) 특허권자\n삼성전기주식회사\n경기도 수원시 영통구 매영로 150 (매탄동)\n(72) 발명자\n이준성\n경기도 수원시 영통구 매영로 150 (매탄동)\n조석현\n경기도 수원시 영통구 매영로 150 (매탄동)\n이용삼\n경기도 수원시 영통구 매영로 150 (매탄동)\n(74) 대리인\n특허법인씨엔에스\n전체 청구항 수 : 총  10  항  심사관 :    최동기\n(54) 발명의 명칭 인쇄회로기판 및 그 제조방법\n(57) 요 약\n본 발명의 인쇄회로기판은, 캐비티를 갖는 코어 기판; 및 상기 캐비티에 수용된 더미칩을 포함하여 구성'

### Chunk 기반 텍스트 분할

In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(document)

### embedding

In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_KEY')

In [9]:
# embedding 패키지 로딩
from langchain_community.embeddings import HuggingFaceHubEmbeddings

In [10]:
# embedding 클래스 정의 
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)


/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(


In [11]:
from langchain.vectorstores import Chroma

### 저장 및 검색

In [12]:
# Chroma DB 에 저장
docsearch = Chroma.from_documents(texts, embeddings)
# retriever 가져옴
retriever = docsearch.as_retriever()

### 프로젝트 템플릿

In [27]:
# from langchain import hub
# rag_prompt = hub.pull("rlm/rag-prompt")
# rag_prompt
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
{context}
넌 특허 전문 변리사 로봇이야.
주어진 질문에 대한 답변을 한글로 답해줘 
{question}
""")

### llm 생성

In [28]:
from langchain_community.chat_models import ChatOllama

In [29]:
model = "mistral-nemo"

In [30]:
llm = ChatOllama(base_url="http://192.168.0.11:11434",
                 model=model,
                 temperature=0)

### RAG Chain 생성

In [31]:
from langchain.schema.runnable import RunnablePassthrough

In [32]:
rag_chain = (
    {
        "context":retriever,
        "question": RunnablePassthrough()
    }
    # |rag_prompt
    | prompt
    | llm    
)

In [33]:
rag_chain.invoke("이 발명의 명칭이 뭐지?")

AIMessage(content='이 발명의 명칭은 "인쇄회로기판 및 그 제조방법"입니다.', response_metadata={'model': 'mistral-nemo', 'created_at': '2024-08-03T15:51:26.135887673Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 703472318, 'load_duration': 18354622, 'prompt_eval_count': 1026, 'prompt_eval_duration': 347306000, 'eval_count': 22, 'eval_duration': 296496000}, id='run-2d8876da-dffb-4aaa-93ab-b9b272294cfc-0')

In [37]:
rag_chain.invoke("이 문서의 발명자, 출원인, 권리자를 추출해서 python 사전 구조로 표시해줘")

AIMessage(content='{\n  "발명자": "미상",\n  "출원인": "미상",\n  "권리자": "미상"\n}', response_metadata={'model': 'mistral-nemo', 'created_at': '2024-08-03T15:52:35.608506517Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 836405155, 'load_duration': 17978612, 'prompt_eval_count': 1026, 'prompt_eval_duration': 339184000, 'eval_count': 32, 'eval_duration': 437233000}, id='run-55109cfc-87c6-4cf2-88ba-db43be4b2a77-0')

In [35]:
rag_chain.invoke("입력된 문서의 수량은 몇개나 되지?")

AIMessage(content='주어진 문서는 총 1개의 문서이다.', response_metadata={'model': 'mistral-nemo', 'created_at': '2024-08-03T15:51:41.190658402Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 564363405, 'load_duration': 18308122, 'prompt_eval_count': 1026, 'prompt_eval_duration': 332729000, 'eval_count': 13, 'eval_duration': 169561000}, id='run-a37dd167-f00a-4a23-9e14-9350400071ff-0')

In [36]:
rag_chain.invoke("입력된 모든 문서의 제목을 표시해줘 ")

AIMessage(content='**제목:** 인쇄회로기판 및 그 제작방법\n\n**내용 요약:**\n\n* 이 특허는 테이퍼 형상으로 가공된 코어 기판(110)과 더미칩(120), 레진층(130), 회로층(140)으로 구성된 인쇄회로기판을 설명한다.\n* 코어 기판(110)은 수지 절연물질로 만들어지며, 에폭시 수지, 폴리이미드 수지 또는 보강재가 함침된 프리프레그가 사용된다.\n* 더미칩(120)은 유리 기판을 레이저 드릴 가공하여 역테이퍼 형상으로 실장한다. 이는 기판 휨 현상을 개선하기 위해 임베디드 기판 특성을 고려한 것이다.\n* 레진층(130)은 열경화성 또는 열가소성 고분자 물질, 세라믹, 유-무기 복합 소재, 글라스 섬유 함침일 수 있으며, 고분자 수지를 포함하는 경우 FR-4, BT(Bismaleimide Triazine), ABF(Ajinomoto Build up Film) 등의 에폭시계 절연 수지를 포함할 수 있다.\n* 코어 기판(110)은 더미칩(120)이 실장될 캐비티(111) 및 두께 방향으로 관통하는 관통홀을 형성한다.', response_metadata={'model': 'mistral-nemo', 'created_at': '2024-08-03T15:51:49.618405821Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4908655081, 'load_duration': 18292913, 'prompt_eval_count': 1026, 'prompt_eval_duration': 334693000, 'eval_count': 319, 'eval_duration': 4511322000}, id='run-be50a4f6-8f0f-4666-be19-1af7e65f4e06-0')